In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier
from xgboost import XGBRegressor
from sklearn.neural_network import MLPClassifier
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
#from tcn import TCN
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from tqdm import tqdm
from sklearn import metrics
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

# Reading the Data
For the initial implementation of the project i use Floodlight dataset and for the other datasets i can use the same methodology.

In [2]:
df_flood = 'complete_dataset_final.csv'
df_flood = pd.read_csv(df_flood)
# Step 1: Create the 'testFull' column
df_flood['testFull'] = df_flood['testCode'] + '_' + df_flood['testMetricCode']
df_flood

,floodlightOpenId,participantCreatedOn_UTC,participantIsControl,participantCountryOfResidence,participantSex,participantBirthYear,participantWeightLbs,participantHeightCms,testName,testCode,...,testMetricCode,testStartedAt_UTC,testEndedAt_UTC,testResultMetricId,testResultMetricCreatedOn_UTC,testResultMetricTimestamp1_UTC,testResultMetricTimestamp2_UTC,testResultMetricValue,algorithmVersion,testFull
0,FL20345491,1/1/2021 1:15:02 AM +00:00,True,AU,female,1958,208,165,Daily Questions,daily_questions,...,mood_response,1/1/2021 10:03:15 PM +00:00,1/1/2021 10:03:23 PM +00:00,424686,1/1/2021 10:03:24 PM +00:00,NaN,NaN,5.00,1,daily_questions_mood_response
1,FL20345491,1/1/2021 1:15:02 AM +00:00,True,AU,female,1958,208,165,Information Processing Speed Baseline Test,ips_baseline,...,correct_responses,1/1/2021 10:05:15 PM +00:00,1/1/2021 10:05:31 PM +00:00,424690,1/1/2021 10:05:31 PM +00:00,NaN,NaN,20.00,1,ips_baseline_correct_responses
2,FL20345491,1/1/2021 1:15:02 AM +00:00,True,AU,female,1958,208,165,Information Processing Speed Baseline Test,ips_baseline,...,response_time_avg,1/1/2021 10:05:15 PM +00:00,1/1/2021 10:05:31 PM +00:00,424691,1/1/2021 10:05:31 PM +00:00,NaN,NaN,0.75,1,ips_baseline_response_time_avg
3,FL20345491,1/1/2021 1:15:02 AM +00:00,True,AU,female,1958,208,165,IPS,ips,...,correct_responses,1/1/2021 10:05:15 PM +00:00,1/1/2021 10:05:31 PM +00:00,424692,1/1/2021 10:05:32 PM +00:00,NaN,NaN,49.00,1,ips_correct_responses
4,FL20345491,1/1/2021 1:15:02 AM +00:00,True,AU,female,1958,208,165,IPS,ips,...,response_time_avg,1/1/2021 10:05:15 PM +00:00,1/1/2021 10:05:31 PM +00:00,424693,1/1/2021 10:05:32 PM +00:00,NaN,NaN,1.84,1,ips_response_time_avg
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
795092,FL96495919,5/3/2022 7:00:38 PM +00:00,False,NL,female,1967,135,180,Mobility,mobility,...,life_space_daily,5/14/2022 10:00:00 PM +00:00,5/15/2022 10:00:00 PM +00:00,190045,5/3/2023 9:48:57 AM +00:00,NaN,NaN,6.62,1,mobility_life_space_daily
795093,FL96495919,5/3/2022 7:00:38 PM +00:00,False,NL,female,1967,135,180,Mobility,mobility,...,life_space_daily,5/24/2022 10:00:00 PM +00:00,5/25/2022 10:00:00 PM +00:00,190046,5/3/2023 9:48:57 AM +00:00,NaN,NaN,3.79,1,mobility_life_space_daily
795094,FL96495919,5/3/2022 7:00:38 PM +00:00,False,NL,female,1967,135,180,Mobility,mobility,...,life_space_daily,5/9/2022 10:00:00 PM +00:00,5/10/2022 10:00:00 PM +00:00,190047,5/3/2023 9:48:57 AM +00:00,NaN,NaN,0.00,1,mobility_life_space_daily
795095,FL96495919,5/3/2022 7:00:38 PM +00:00,False,NL,female,1967,135,180,Mobility,mobility,...,life_space_daily,6/29/2022 10:00:00 PM +00:00,6/30/2022 10:00:00 PM +00:00,190049,5/3/2023 9:49:18 AM +00:00,NaN,NaN,0.00,1,mobility_life_space_daily


In [3]:
# Calculate participant's age and test duration
df_flood['participantCreatedOn_UTC'] = pd.to_datetime(df_flood['participantCreatedOn_UTC'], utc=True)
df_flood['created_year'] = df_flood['participantCreatedOn_UTC'].dt.year
df_flood['participantAge'] = df_flood['created_year'] - df_flood['participantBirthYear']
df_flood

,floodlightOpenId,participantCreatedOn_UTC,participantIsControl,participantCountryOfResidence,participantSex,participantBirthYear,participantWeightLbs,participantHeightCms,testName,testCode,...,testEndedAt_UTC,testResultMetricId,testResultMetricCreatedOn_UTC,testResultMetricTimestamp1_UTC,testResultMetricTimestamp2_UTC,testResultMetricValue,algorithmVersion,testFull,created_year,participantAge
0,FL20345491,2021-01-01 01:15:02+00:00,True,AU,female,1958,208,165,Daily Questions,daily_questions,...,1/1/2021 10:03:23 PM +00:00,424686,1/1/2021 10:03:24 PM +00:00,NaN,NaN,5.00,1,daily_questions_mood_response,2021,63
1,FL20345491,2021-01-01 01:15:02+00:00,True,AU,female,1958,208,165,Information Processing Speed Baseline Test,ips_baseline,...,1/1/2021 10:05:31 PM +00:00,424690,1/1/2021 10:05:31 PM +00:00,NaN,NaN,20.00,1,ips_baseline_correct_responses,2021,63
2,FL20345491,2021-01-01 01:15:02+00:00,True,AU,female,1958,208,165,Information Processing Speed Baseline Test,ips_baseline,...,1/1/2021 10:05:31 PM +00:00,424691,1/1/2021 10:05:31 PM +00:00,NaN,NaN,0.75,1,ips_baseline_response_time_avg,2021,63
3,FL20345491,2021-01-01 01:15:02+00:00,True,AU,female,1958,208,165,IPS,ips,...,1/1/2021 10:05:31 PM +00:00,424692,1/1/2021 10:05:32 PM +00:00,NaN,NaN,49.00,1,ips_correct_responses,2021,63
4,FL20345491,2021-01-01 01:15:02+00:00,True,AU,female,1958,208,165,IPS,ips,...,1/1/2021 10:05:31 PM +00:00,424693,1/1/2021 10:05:32 PM +00:00,NaN,NaN,1.84,1,ips_response_time_avg,2021,63
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
795092,FL96495919,2022-05-03 19:00:38+00:00,False,NL,female,1967,135,180,Mobility,mobility,...,5/15/2022 10:00:00 PM +00:00,190045,5/3/2023 9:48:57 AM +00:00,NaN,NaN,6.62,1,mobility_life_space_daily,2022,55
795093,FL96495919,2022-05-03 19:00:38+00:00,False,NL,female,1967,135,180,Mobility,mobility,...,5/25/2022 10:00:00 PM +00:00,190046,5/3/2023 9:48:57 AM +00:00,NaN,NaN,3.79,1,mobility_life_space_daily,2022,55
795094,FL96495919,2022-05-03 19:00:38+00:00,False,NL,female,1967,135,180,Mobility,mobility,...,5/10/2022 10:00:00 PM +00:00,190047,5/3/2023 9:48:57 AM +00:00,NaN,NaN,0.00,1,mobility_life_space_daily,2022,55
795095,FL96495919,2022-05-03 19:00:38+00:00,False,NL,female,1967,135,180,Mobility,mobility,...,6/30/2022 10:00:00 PM +00:00,190049,5/3/2023 9:49:18 AM +00:00,NaN,NaN,0.00,1,mobility_life_space_daily,2022,55


In [4]:
df_flood.columns

Index(['floodlightOpenId', 'participantCreatedOn_UTC', 'participantIsControl',
       'participantCountryOfResidence', 'participantSex',
       'participantBirthYear', 'participantWeightLbs', 'participantHeightCms',
       'testName', 'testCode', 'testMetricName', 'testMetricCode',
       'testStartedAt_UTC', 'testEndedAt_UTC', 'testResultMetricId',
       'testResultMetricCreatedOn_UTC', 'testResultMetricTimestamp1_UTC',
       'testResultMetricTimestamp2_UTC', 'testResultMetricValue',
       'algorithmVersion', 'testFull', 'created_year', 'participantAge'],
      dtype='object')

In [13]:
demog = ['floodlightOpenId','participantIsControl','participantSex','participantWeightLbs','participantHeightCms','participantAge']
demographic = df_flood[demog]
# demog should be with unique floodlightOpenId
demographic = demographic.drop_duplicates(subset='floodlightOpenId')
demographic

,floodlightOpenId,participantIsControl,participantSex,participantWeightLbs,participantHeightCms,participantAge
0,FL20345491,True,female,208,165,63
1589,FL95672089,False,female,208,158,60
1617,FL27294513,True,female,164,162,39
1688,FL71538506,True,female,307,173,41
1717,FL27887817,False,female,468,158,38
...,...,...,...,...,...,...
773065,FL61994596,True,male,180,175,29
773084,FL18785106,False,female,183,175,65
773491,FL68195789,True,female,148,170,30
780084,FL99125103,False,female,113,161,64


# Unique flood Id

In [14]:
df_flood['floodlightOpenId'].nunique()

2624

In [15]:
# Group by 'floodlightOpenId' and 'testFull' and calculate the mean of 'testResultMetricValue'
df_flood_grouped = df_flood.groupby(['floodlightOpenId', 'testFull'])['testResultMetricValue'].mean().reset_index()
df_flood_grouped

,floodlightOpenId,testFull,testResultMetricValue
0,FL00088674,daily_questions_mood_response,4.00
1,FL00088674,draw_a_shape_bottom_to_top_hausdorff_distance_...,0.13
2,FL00088674,draw_a_shape_circle_hausdorff_distance_best,0.14
3,FL00088674,draw_a_shape_figure8_hausdorff_distance_best,0.12
4,FL00088674,draw_a_shape_hand_used,1.00
...,...,...,...
46229,FL99985016,ips_correct_responses,47.00
46230,FL99985016,ips_response_time_avg,1.91
46231,FL99985016,pinching_hand_used,1.00
46232,FL99985016,pinching_successful_pinches,12.00


In [16]:
# merge the demographic data with the grouped data
df_flood_grouped = pd.merge(df_flood_grouped, demographic, on='floodlightOpenId')
df_flood_grouped

,floodlightOpenId,testFull,testResultMetricValue,participantIsControl,participantSex,participantWeightLbs,participantHeightCms,participantAge
0,FL00088674,daily_questions_mood_response,4.00,False,male,245,188,38
1,FL00088674,draw_a_shape_bottom_to_top_hausdorff_distance_...,0.13,False,male,245,188,38
2,FL00088674,draw_a_shape_circle_hausdorff_distance_best,0.14,False,male,245,188,38
3,FL00088674,draw_a_shape_figure8_hausdorff_distance_best,0.12,False,male,245,188,38
4,FL00088674,draw_a_shape_hand_used,1.00,False,male,245,188,38
...,...,...,...,...,...,...,...,...
46229,FL99985016,ips_correct_responses,47.00,False,female,243,176,42
46230,FL99985016,ips_response_time_avg,1.91,False,female,243,176,42
46231,FL99985016,pinching_hand_used,1.00,False,female,243,176,42
46232,FL99985016,pinching_successful_pinches,12.00,False,female,243,176,42


In [17]:
# Now Group by both 'floodlightOpenId' and 'testFull' and calculate the mean of 'testResultMetricValue'
df_flood_grouped = df_flood_grouped.groupby(['floodlightOpenId', 'testFull']).mean().reset_index()
df_flood_grouped

/var/folders/qh/bhv0wgwn7mn02b3mgp7vztnw0000gn/T/ipykernel_50277/2002974245.py:2: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df_flood_grouped = df_flood_grouped.groupby(['floodlightOpenId', 'testFull']).mean().reset_index()


,floodlightOpenId,testFull,testResultMetricValue,participantIsControl,participantWeightLbs,participantHeightCms,participantAge
0,FL00088674,daily_questions_mood_response,4.00,0.0,245.0,188.0,38.0
1,FL00088674,draw_a_shape_bottom_to_top_hausdorff_distance_...,0.13,0.0,245.0,188.0,38.0
2,FL00088674,draw_a_shape_circle_hausdorff_distance_best,0.14,0.0,245.0,188.0,38.0
3,FL00088674,draw_a_shape_figure8_hausdorff_distance_best,0.12,0.0,245.0,188.0,38.0
4,FL00088674,draw_a_shape_hand_used,1.00,0.0,245.0,188.0,38.0
...,...,...,...,...,...,...,...
46229,FL99985016,ips_correct_responses,47.00,0.0,243.0,176.0,42.0
46230,FL99985016,ips_response_time_avg,1.91,0.0,243.0,176.0,42.0
46231,FL99985016,pinching_hand_used,1.00,0.0,243.0,176.0,42.0
46232,FL99985016,pinching_successful_pinches,12.00,0.0,243.0,176.0,42.0


In [18]:
# now create a pivot table that has 'floodlightOpenId' as the index, 'testFull' as the columns and 'testResultMetricValue' as the values
df_flood_pivot = df_flood_grouped.pivot(index='floodlightOpenId', columns='testFull', values='testResultMetricValue')
df_flood_pivot


testFull,daily_questions_mood_response,draw_a_shape_bottom_to_top_hausdorff_distance_best,draw_a_shape_circle_hausdorff_distance_best,draw_a_shape_figure8_hausdorff_distance_best,draw_a_shape_hand_used,draw_a_shape_mean_hausdorff_distance_best,draw_a_shape_number_shapes_drawn_correctly,draw_a_shape_spiral_hausdorff_distance_best,draw_a_shape_square_hausdorff_distance_best,draw_a_shape_top_to_bottom_hausdorff_distance_best,...,five_u_turn_turns,ips_baseline_correct_responses,ips_baseline_response_time_avg,ips_correct_responses,ips_response_time_avg,mobility_life_space_daily,pinching_hand_used,pinching_successful_pinches,static_balance_sway_path,two_min_walk_steps
floodlightOpenId,,,,,,,,,,,,,,,,,,,,,
FL00088674,4.000000,0.130000,0.140000,0.120000,1.000000,0.150000,2.000000,0.170000,0.230000,0.140000,...,10.000000,24.0,0.63,46.0,1.960000,0.000000,1.000000,36.00,204.630000,NaN
FL00128001,3.250000,0.088571,0.085714,0.082857,0.500000,0.102857,3.500000,0.120000,0.140000,0.110000,...,9.833333,20.5,0.73,51.0,1.790000,6.486800,0.500000,42.25,126.013333,NaN
FL00147029,3.000000,0.080000,0.075000,0.070000,1.000000,0.095000,6.000000,0.105000,0.130000,0.090000,...,11.000000,21.0,0.71,52.0,1.730000,0.000000,1.000000,53.00,92.470000,236.0
FL00229560,4.333333,0.140000,0.116667,0.113333,0.666667,0.150000,2.333333,0.156667,0.233333,0.130000,...,6.000000,20.0,0.75,56.5,1.595000,NaN,0.666667,54.00,15.160000,NaN
FL00277926,2.900000,0.098235,0.121765,0.094706,0.500000,0.116471,2.700000,0.134118,0.155882,0.095882,...,10.555556,15.0,1.00,35.0,2.573333,40.941127,0.500000,13.50,11.222000,224.8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
FL99819582,3.000000,0.120000,0.110000,0.110000,1.000000,0.140000,5.000000,0.170000,0.210000,0.140000,...,0.000000,21.0,0.71,48.0,1.880000,NaN,1.000000,26.00,47.200000,NaN
FL99840724,5.000000,0.130000,0.110000,0.110000,1.000000,0.140000,3.000000,0.140000,0.210000,0.140000,...,NaN,20.0,0.75,62.0,1.450000,0.000000,1.000000,39.00,NaN,NaN
FL99922020,4.000000,0.220000,0.130000,0.110000,1.000000,0.370000,3.000000,0.150000,0.870000,0.760000,...,9.500000,22.0,702.27,59.0,1532.900000,NaN,1.000000,19.00,141.480000,NaN


In [19]:
# now merge to see if the demographic data is still there
df_flood_pivot = pd.merge(df_flood_pivot, demographic, on='floodlightOpenId')
df_flood_pivot

,floodlightOpenId,daily_questions_mood_response,draw_a_shape_bottom_to_top_hausdorff_distance_best,draw_a_shape_circle_hausdorff_distance_best,draw_a_shape_figure8_hausdorff_distance_best,draw_a_shape_hand_used,draw_a_shape_mean_hausdorff_distance_best,draw_a_shape_number_shapes_drawn_correctly,draw_a_shape_spiral_hausdorff_distance_best,draw_a_shape_square_hausdorff_distance_best,...,mobility_life_space_daily,pinching_hand_used,pinching_successful_pinches,static_balance_sway_path,two_min_walk_steps,participantIsControl,participantSex,participantWeightLbs,participantHeightCms,participantAge
0,FL00088674,4.000000,0.130000,0.140000,0.120000,1.000000,0.150000,2.000000,0.170000,0.230000,...,0.000000,1.000000,36.00,204.630000,NaN,False,male,245,188,38
1,FL00128001,3.250000,0.088571,0.085714,0.082857,0.500000,0.102857,3.500000,0.120000,0.140000,...,6.486800,0.500000,42.25,126.013333,NaN,False,female,181,170,25
2,FL00147029,3.000000,0.080000,0.075000,0.070000,1.000000,0.095000,6.000000,0.105000,0.130000,...,0.000000,1.000000,53.00,92.470000,236.0,True,female,206,168,49
3,FL00229560,4.333333,0.140000,0.116667,0.113333,0.666667,0.150000,2.333333,0.156667,0.233333,...,NaN,0.666667,54.00,15.160000,NaN,True,male,199,182,39
4,FL00277926,2.900000,0.098235,0.121765,0.094706,0.500000,0.116471,2.700000,0.134118,0.155882,...,40.941127,0.500000,13.50,11.222000,224.8,False,male,187,178,73
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2619,FL99819582,3.000000,0.120000,0.110000,0.110000,1.000000,0.140000,5.000000,0.170000,0.210000,...,NaN,1.000000,26.00,47.200000,NaN,True,female,111,168,0
2620,FL99840724,5.000000,0.130000,0.110000,0.110000,1.000000,0.140000,3.000000,0.140000,0.210000,...,0.000000,1.000000,39.00,NaN,NaN,True,male,220,163,38
2621,FL99922020,4.000000,0.220000,0.130000,0.110000,1.000000,0.370000,3.000000,0.150000,0.870000,...,NaN,1.000000,19.00,141.480000,NaN,False,female,94,158,36
2622,FL99968202,3.000000,0.070000,0.070000,0.075000,1.000000,0.090000,3.000000,0.095000,0.115000,...,338.509515,1.000000,28.00,83.900000,NaN,True,male,175,187,33


# You Should Deal with Null Values First 

In [22]:
# count NAN in X
df_flood_pivot.isna().sum()

floodlightOpenId                                         0
daily_questions_mood_response                          143
draw_a_shape_bottom_to_top_hausdorff_distance_best     334
draw_a_shape_circle_hausdorff_distance_best            334
draw_a_shape_figure8_hausdorff_distance_best           333
draw_a_shape_hand_used                                 320
draw_a_shape_mean_hausdorff_distance_best              319
draw_a_shape_number_shapes_drawn_correctly             320
draw_a_shape_spiral_hausdorff_distance_best            336
draw_a_shape_square_hausdorff_distance_best            334
draw_a_shape_top_to_bottom_hausdorff_distance_best     333
five_u_turn_turn_speed_avg                             735
five_u_turn_turns                                      735
ips_baseline_correct_responses                         281
ips_baseline_response_time_avg                         281
ips_correct_responses                                  273
ips_response_time_avg                                  2

# Then you Can start ML Process

In [ ]:
# get participantIsControl as a target
y = df_flood_pivot['participantIsControl']
X = df_flood_pivot.drop(columns=['participantIsControl'])

# drop 'floodlightOpenId' 
X = X.drop(columns=['floodlightOpenId'])

# make participantSex a one hot encoded variable
X = pd.get_dummies(X, columns=['participantSex'])

# split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# scale the data
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# create a logistic regression model
model = LogisticRegression()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

# calculate the accuracy
accuracy = metrics.accuracy_score(y_test, y_pred)
accuracy